In [475]:
import pandas as pd

In [477]:
matches = pd.read_csv("matches.csv", index_col=0)

In [479]:
matches["round"].value_counts()

round
Matchweek 1     108
Matchweek 14    108
Matchweek 24    108
Matchweek 23    108
Matchweek 22    108
Matchweek 21    108
Matchweek 20    108
Matchweek 19    108
Matchweek 2     108
Matchweek 17    108
Matchweek 16    108
Matchweek 15    108
Matchweek 18    108
Matchweek 13    108
Matchweek 7     108
Matchweek 12    108
Matchweek 4     108
Matchweek 5     108
Matchweek 6     108
Matchweek 3     108
Matchweek 8     108
Matchweek 9     108
Matchweek 10    108
Matchweek 11    108
Matchweek 25    104
Matchweek 26     90
Matchweek 27     90
Matchweek 28     90
Matchweek 29     90
Matchweek 30     90
Matchweek 31     90
Matchweek 32     90
Matchweek 33     90
Matchweek 34     90
Name: count, dtype: int64

In [481]:
matches.dtypes

date              object
time              object
comp              object
round             object
day               object
venue             object
result            object
gf               float64
ga               float64
opponent          object
xg               float64
xga              float64
poss             float64
attendance       float64
captain           object
formation         object
opp formation     object
referee           object
match report      object
notes             object
sh               float64
sot              float64
dist             float64
fk               float64
pk                 int64
pkatt              int64
season             int64
team              object
dtype: object

In [483]:
del matches["opp formation"]
matches["date"] = pd.to_datetime(matches["date"])

In [485]:
matches.shape

(3506, 27)

In [487]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek
matches["target"] = (matches["result"] == "W").astype("int")

In [489]:
class MissingDictMatches(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Bayer Leverkusen": "Leverkusen", 
    "Eintracht Frankfurt": "Eint Frankfurt", 
    "Monchengladbach": "Gladbach", 
    "St Pauli": "St. Pauli"
} 
mapping = MissingDictMatches(**map_values)

In [491]:
matches[["opponent", "team"]] = matches[["opponent", "team"]].map(lambda x: mapping[x])

In [493]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = {}

    for col in cols:
        if col == "target":  # Falls die Spalte "target" ist, nimm die Summe
            rolling_stats[col] = group[col].rolling(4, closed="left").sum()
        else:  # Für alle anderen Spalten: Rolling-Mittelwert berechnen
            rolling_stats[col] = group[col].rolling(4, closed="left").mean()
    
    rolling_df = pd.DataFrame(rolling_stats)
    group[new_cols] = rolling_df.values
    group = group.dropna(subset=new_cols)
    return group

In [495]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt", "target"]
new_cols = [f"{c}_rolling" for c in cols]

matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

/var/folders/w5/9bdt5y9j6ts7mdp7kvnhvr140000gn/T/ipykernel_50295/277178271.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [498]:
matches_rolling = matches_rolling.droplevel('team')

In [500]:
matches_rolling.index = range(matches_rolling.shape[0])

In [502]:
def get_season(date):
    year = date.year
    if date.month >= 8: 
        return year + 1
    return year

In [504]:
matches_rolling["season"] = matches_rolling["date"].apply(get_season)

In [506]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,target_rolling
0,2020-10-25,15:30,Bundesliga,Matchweek 5,Sun,Away,L,1.0,2.0,Wolfsburg,...,0,0.75,1.50,9.75,3.75,21.625,0.25,0.00,0.00,1.0
1,2020-10-31,15:30,Bundesliga,Matchweek 6,Sat,Home,L,0.0,2.0,Dortmund,...,0,0.75,1.75,9.00,3.00,22.000,0.50,0.00,0.00,1.0
2,2020-11-07,15:30,Bundesliga,Matchweek 7,Sat,Away,L,0.0,5.0,Union Berlin,...,0,0.50,2.25,8.50,3.00,20.400,0.50,0.00,0.00,0.0
3,2020-11-21,15:30,Bundesliga,Matchweek 8,Sat,Home,L,1.0,2.0,Leverkusen,...,0,0.50,3.25,7.75,2.25,20.900,0.75,0.00,0.00,0.0
4,2020-12-19,15:30,Bundesliga,Matchweek 13,Sat,Away,W,1.0,0.0,Schalke 04,...,1,0.75,1.50,10.75,3.00,19.050,0.00,0.00,0.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3385,2025-02-08,15:30,Bundesliga,Matchweek 21,Sat,Home,D,0.0,0.0,Leverkusen,...,0,2.50,1.75,13.75,3.50,17.150,0.25,0.25,0.25,1.0
3386,2025-02-15,15:30,Bundesliga,Matchweek 22,Sat,Away,W,2.0,1.0,Stuttgart,...,1,1.25,1.50,13.25,4.50,17.825,0.25,0.00,0.00,0.0
3387,2025-02-22,15:30,Bundesliga,Matchweek 23,Sat,Home,D,1.0,1.0,Bochum,...,0,1.25,1.00,13.00,4.25,18.300,0.25,0.25,0.25,1.0
3388,2025-03-01,15:30,Bundesliga,Matchweek 24,Sat,Away,W,2.0,1.0,Werder Bremen,...,1,1.00,0.75,10.75,4.50,18.300,0.25,0.25,0.25,1.0


In [508]:
matches_rolling.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team', 'venue_code', 'opp_code', 'hour', 'day_code',
       'target', 'gf_rolling', 'ga_rolling', 'sh_rolling', 'sot_rolling',
       'dist_rolling', 'fk_rolling', 'pk_rolling', 'pkatt_rolling',
       'target_rolling'],
      dtype='object')

In [510]:
import os
import glob

In [512]:
table_csv_files = glob.glob(os.path.join("tables/", "*.csv"))

In [514]:
tables = [pd.read_csv(file) for file in table_csv_files]

In [516]:
tables_combined = pd.concat(tables, ignore_index=True)

In [518]:
tables_combined.columns

Index(['Unnamed: 0', 'team', 'rank', 'goal_diff', 'points', 'season'], dtype='object')

In [520]:
tables_combined["team"].unique()

array(['Bayern', 'RB Leipzig', 'Dortmund', 'Wolfsburg', 'Frankfurt',
       'Leverkusen', 'Union Berlin', "M'gladbach", 'Stuttgart',
       'Freiburg', 'Hoffenheim', 'Mainz', 'Augsburg', 'Hertha BSC',
       'Bielefeld', 'Köln', 'Bremen', 'Schalke', 'Düsseldorf',
       'Paderborn', 'VfL Bochum', 'Fürth', 'Heidenheim', 'Darmstadt',
       'Hannover', 'Hamburg', 'Nürnberg'], dtype=object)

In [522]:
matches_rolling["team"].unique()

array(['Arminia', 'Augsburg', 'Bayern Munich', 'Bochum', 'Darmstadt 98',
       'Dortmund', 'Dusseldorf', 'Eint Frankfurt', 'Freiburg', 'Gladbach',
       'Hannover 96', 'Heidenheim', 'Hertha BSC', 'Hoffenheim',
       'Holstein Kiel', 'Koln', 'Leverkusen', 'Mainz 05', 'Nurnberg',
       'Paderborn 07', 'RB Leipzig', 'Schalke 04', 'St. Pauli',
       'Stuttgart', 'Union Berlin', 'Werder Bremen', 'Wolfsburg'],
      dtype=object)

In [524]:
class MissingDictTables(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Bayern": "Bayern Munich",
    "Frankfurt": "Eint Frankfurt",
    "M'gladbach": "Gladbach",
    "Mainz": "Mainz 05",
    "Bielefeld": "Arminia",
    "Bremen": "Werder Bremen",
    "Schalke": "Schalke 04",
    "Paderborn": "Paderborn 07",
    "VfL Bochum": "Bochum",
    "Darmstadt": "Darmstadt 98",
    "Hannover": "Hannover 96"    
} 
mapping = MissingDictTables(**map_values)

In [526]:
tables_combined[["team"]] = tables_combined[["team"]].map(lambda x: mapping[x])

In [528]:
tables_combined = tables_combined.drop(columns=["Unnamed: 0", "goal_diff", "points"])

In [530]:
tables_combined.rename(columns={"season": "season_ranking"}, inplace=True)

In [532]:
matches_rolling["last_season"] = matches_rolling["season"] - 1

In [534]:
matches_rolling = matches_rolling.merge(
    tables_combined, left_on=["last_season", "team"], right_on=["season_ranking", "team"], how="left"
)

In [536]:
matches_rolling = matches_rolling.merge(
    tables_combined, left_on=["last_season", "opponent"], right_on=["season_ranking", "team"], how="left", suffixes=("", "_op")
)

In [538]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk_rolling,pk_rolling,pkatt_rolling,target_rolling,last_season,rank,season_ranking,team_op,rank_op,season_ranking_op
0,2020-10-25,15:30,Bundesliga,Matchweek 5,Sun,Away,L,1.0,2.0,Wolfsburg,...,0.25,0.00,0.00,1.0,2020,NaN,NaN,Wolfsburg,7.0,2020.0
1,2020-10-31,15:30,Bundesliga,Matchweek 6,Sat,Home,L,0.0,2.0,Dortmund,...,0.50,0.00,0.00,1.0,2020,NaN,NaN,Dortmund,2.0,2020.0
2,2020-11-07,15:30,Bundesliga,Matchweek 7,Sat,Away,L,0.0,5.0,Union Berlin,...,0.50,0.00,0.00,0.0,2020,NaN,NaN,Union Berlin,11.0,2020.0
3,2020-11-21,15:30,Bundesliga,Matchweek 8,Sat,Home,L,1.0,2.0,Leverkusen,...,0.75,0.00,0.00,0.0,2020,NaN,NaN,Leverkusen,5.0,2020.0
4,2020-12-19,15:30,Bundesliga,Matchweek 13,Sat,Away,W,1.0,0.0,Schalke 04,...,0.00,0.00,0.00,1.0,2020,NaN,NaN,Schalke 04,12.0,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3385,2025-02-08,15:30,Bundesliga,Matchweek 21,Sat,Home,D,0.0,0.0,Leverkusen,...,0.25,0.25,0.25,1.0,2024,12.0,2024.0,Leverkusen,1.0,2024.0
3386,2025-02-15,15:30,Bundesliga,Matchweek 22,Sat,Away,W,2.0,1.0,Stuttgart,...,0.25,0.00,0.00,0.0,2024,12.0,2024.0,Stuttgart,2.0,2024.0
3387,2025-02-22,15:30,Bundesliga,Matchweek 23,Sat,Home,D,1.0,1.0,Bochum,...,0.25,0.25,0.25,1.0,2024,12.0,2024.0,Bochum,16.0,2024.0
3388,2025-03-01,15:30,Bundesliga,Matchweek 24,Sat,Away,W,2.0,1.0,Werder Bremen,...,0.25,0.25,0.25,1.0,2024,12.0,2024.0,Werder Bremen,9.0,2024.0


In [540]:
matches_rolling.drop(columns=["season_ranking", "season_ranking_op", "team_op", "last_season"], inplace=True)

In [542]:
matches_rolling["rank"] = matches_rolling["rank"].fillna(19)
matches_rolling["rank_op"] = matches_rolling["rank_op"].fillna(19)

In [544]:
value_csv_files = glob.glob(os.path.join("team_values/", "*.csv"))

In [546]:
values = [pd.read_csv(file) for file in value_csv_files]

In [548]:
values_combined = pd.concat(values, ignore_index=True)

In [550]:
values_combined.columns

Index(['Unnamed: 0', 'team', 'mean_age', 'mean_value', 'season'], dtype='object')

In [552]:
matches_rolling["team"].unique()

array(['Arminia', 'Augsburg', 'Bayern Munich', 'Bochum', 'Darmstadt 98',
       'Dortmund', 'Dusseldorf', 'Eint Frankfurt', 'Freiburg', 'Gladbach',
       'Hannover 96', 'Heidenheim', 'Hertha BSC', 'Hoffenheim',
       'Holstein Kiel', 'Koln', 'Leverkusen', 'Mainz 05', 'Nurnberg',
       'Paderborn 07', 'RB Leipzig', 'Schalke 04', 'St. Pauli',
       'Stuttgart', 'Union Berlin', 'Werder Bremen', 'Wolfsburg'],
      dtype=object)

In [554]:
values_combined["team"].unique()

array(['FC Bayern München', 'Bayer 04 Leverkusen', 'RB Leipzig',
       'Borussia Dortmund', 'VfB Stuttgart', 'Eintracht Frankfurt',
       'VfL Wolfsburg', 'SC Freiburg', 'TSG 1899 Hoffenheim',
       'Borussia Mönchengladbach', '1.FC Union Berlin', '1.FSV Mainz 05',
       'FC Augsburg', 'SV Werder Bremen', '1.FC Köln',
       '1.FC Heidenheim 1846', 'VfL Bochum', 'SV Darmstadt 98',
       'FC St. Pauli', 'Holstein Kiel', 'FC Schalke 04', 'Hertha BSC',
       'Fortuna Düsseldorf', 'Hannover 96', '1.FC Nürnberg',
       'Arminia Bielefeld', 'SpVgg Greuther Fürth', 'SC Paderborn 07'],
      dtype=object)

In [556]:
class MissingDictValues(dict):
    __missing__ = lambda self, key: key

map_values = {
    "FC Bayern München": "Bayern Munich",
    "Bayer 04 Leverkusen": "Leverkusen",
    "Borussia Dortmund": "Dortmund",
    "VfB Stuttgart": "Stuttgart",
    "Eintracht Frankfurt": "Eint Frankfurt",
    "VfL Wolfsburg": "Wolfsburg",
    "SC Freiburg": "Freiburg",
    "TSG 1899 Hoffenheim": "Hoffenheim",
    "Borussia Mönchengladbach": "Gladbach",
    "1.FC Union Berlin": "Union Berlin",
    "1.FSV Mainz 05": "Mainz 05",
    "FC Augsburg": "Augsburg",
    "SV Werder Bremen": "Werder Bremen",
    "1.FC Köln": "Köln",
    "1.FC Heidenheim 1846": "Heidenheim",
    "VfL Bochum": "Bochum",
    "SV Darmstadt 98": "Darmstadt 98",
    "FC St. Pauli": "St. Pauli",
    "Arminia Bielefeld": "Arminia",
    "FC Schalke 04": "Schalke 04",
    "Fortuna Düsseldorf": "Düsseldorf",
    "1.FC Nürnberg": "Nürnberg",
    "SC Paderborn 07": "Paderborn 07"
} 
mapping = MissingDictValues(**map_values)

In [558]:
values_combined[["team"]] = values_combined[["team"]].map(lambda x: mapping[x])

In [560]:
values_combined["team"].unique()

array(['Bayern Munich', 'Leverkusen', 'RB Leipzig', 'Dortmund',
       'Stuttgart', 'Eint Frankfurt', 'Wolfsburg', 'Freiburg',
       'Hoffenheim', 'Gladbach', 'Union Berlin', 'Mainz 05', 'Augsburg',
       'Werder Bremen', 'Köln', 'Heidenheim', 'Bochum', 'Darmstadt 98',
       'St. Pauli', 'Holstein Kiel', 'Schalke 04', 'Hertha BSC',
       'Düsseldorf', 'Hannover 96', 'Nürnberg', 'Arminia',
       'SpVgg Greuther Fürth', 'Paderborn 07'], dtype=object)

In [562]:
values_combined = values_combined.drop(columns=["Unnamed: 0"])

In [564]:
matches_rolling.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team', 'venue_code', 'opp_code', 'hour', 'day_code',
       'target', 'gf_rolling', 'ga_rolling', 'sh_rolling', 'sot_rolling',
       'dist_rolling', 'fk_rolling', 'pk_rolling', 'pkatt_rolling',
       'target_rolling', 'rank', 'rank_op'],
      dtype='object')

In [566]:
matches_rolling = matches_rolling.merge(values_combined, on=["season", "team"], how="left")

In [568]:
matches_rolling = matches_rolling.merge(values_combined, left_on=["season", "opponent"], right_on=["season", "team"], how="left", suffixes=("", "_op"))

In [570]:
matches_rolling.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team', 'venue_code', 'opp_code', 'hour', 'day_code',
       'target', 'gf_rolling', 'ga_rolling', 'sh_rolling', 'sot_rolling',
       'dist_rolling', 'fk_rolling', 'pk_rolling', 'pkatt_rolling',
       'target_rolling', 'rank', 'rank_op', 'mean_age', 'mean_value',
       'team_op', 'mean_age_op', 'mean_value_op'],
      dtype='object')

In [572]:
matches_rolling.drop(columns=["team_op"], inplace=True)

In [574]:
from sklearn.ensemble import RandomForestClassifier

In [576]:
rf = RandomForestClassifier(n_estimators=200, min_samples_split=30, random_state=1)

In [578]:
train = matches_rolling[matches_rolling["date"] < '2024-08-01'] # previous seasons

In [580]:
test = matches_rolling[matches_rolling["date"] > '2024-08-01'] # current season

In [582]:
predictors = [
    "venue_code", 
    "opp_code", 
    "hour",
    "day_code", 
    "gf_rolling", 
    "ga_rolling", 
    "sh_rolling", 
    "sot_rolling", 
    "dist_rolling", 
    "fk_rolling", 
    "pk_rolling", 
    "pkatt_rolling",
    "target_rolling",
    "rank",
    "rank_op",
    "mean_age",
    "mean_value",
    "mean_age_op",
    "mean_value_op"
    
]

In [584]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=30, n_estimators=200, random_state=1)

In [586]:
preds = rf.predict(test[predictors])

In [588]:
from sklearn.metrics import accuracy_score

In [590]:
acc = accuracy_score(test["target"], preds)

In [592]:
acc

0.6820276497695853

In [594]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))

In [596]:
combined

,actual,predicted
192,0,0
193,0,0
194,1,1
195,0,0
196,0,0
...,...,...
3385,0,0
3386,1,0
3387,0,1
3388,1,0


In [598]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,222,43
1,95,74


In [600]:
from sklearn.metrics import precision_score

precision_score(test["target"], preds)

0.6324786324786325